In [ ]:
%install '.package(path: "$cwd/FastaiNotebooks")' FastaiNotebooks

Installing packages:
	.package(path: "/home/ubuntu/notebooks/swift/FastaiNotebooks")
		FastaiNotebooks
Working in: /tmp/tmpxgu193tm
Fetching https://github.com/mxcl/Path.swift
Fetching https://github.com/JustHTTP/Just
Completed resolution in 1.67s
Cloning https://github.com/JustHTTP/Just
Resolving https://github.com/JustHTTP/Just at 0.7.1
Cloning https://github.com/mxcl/Path.swift
Resolving https://github.com/mxcl/Path.swift at 0.16.2
Compile Swift Module 'Just' (1 sources)
Compile Swift Module 'Path' (9 sources)
Compile Swift Module 'FastaiNotebooks' (3 sources)
Compile Swift Module 'jupyterInstalledPackages' (1 sources)
Linking ./.build/x86_64-unknown-linux/debug/libjupyterInstalledPackages.so
Installation complete!

In [ ]:
import FastaiNotebooks

In [ ]:
// export
import Path
import TensorFlow

## The forward and backward passes

In [ ]:
let mnist = MnistDataset(path: Path.home/".fastai"/"data"/"mnist_tst")
var x_train = mnist.xTrain
var y_train = mnist.yTrain
var x_valid = mnist.xValid
var y_valid = mnist.yValid

In [ ]:
let (n,m) = (Int(x_train.shape[0]),Int(x_train.shape[1]))
let c = y_train.max()+1

In [ ]:
let nh = 50

In [ ]:
struct MyModel: Layer {
    var layer1 = Dense<Float>(inputSize: m, outputSize: nh, activation: relu)
    var layer2 = Dense<Float>(inputSize: nh, outputSize: 1)
    
    @differentiable
    func applied(to input: Tensor<Float>, in context: Context) -> Tensor<Float> {
        return input.sequenced(in: context, through: layer1, layer2)
    }
}

In [ ]:
var model = MyModel()

In [ ]:
let trainingContext = Context(learningPhase: .training)

In [ ]:
let y_trainf = Tensor<Float>(y_train)

In [ ]:
let optimizer = SGD<MyModel, Float>(learningRate: 0.01)

In [ ]:
struct Batch: TensorGroup{
    let x: Tensor<Float>
    let y: Tensor<Float>
    
    static var _typeList: [TensorDataType] = [
        Float.tensorFlowDataType,
        Float.tensorFlowDataType
    ]
    static var _unknownShapeList: [TensorShape?] = [nil, nil]
    func _unpackTensorHandles(into address: UnsafeMutablePointer<CTensorHandle>?) {
        address!.advanced(by: 0).initialize(to: x.handle._cTensorHandle)
        address!.advanced(by: 1).initialize(to: y.handle._cTensorHandle)
    }
    init(_owning tensorHandles: UnsafePointer<CTensorHandle>?) {
        x = Tensor(handle: TensorHandle(_owning: tensorHandles!.advanced(by: 0).pointee))
        y = Tensor(handle: TensorHandle(_owning: tensorHandles!.advanced(by: 1).pointee))
    }
    
    init(x: Tensor<Float>, y: Tensor<Float>){
        self.x = x
        self.y = y
    }
}

In [ ]:
let train_ds:Dataset<Batch> = Dataset(elements:Batch(x:x_train, y:y_trainf)).batched(64)

In [ ]:
func basic_training_loop<Model, Opt:Optimizer> (train_ds:Dataset<Batch>, model: inout Model, opt: inout Opt, 
                         loss_func: @differentiable (Tensor<Float>, Tensor<Float>)->Tensor<Float>)
                         where Opt.Model == Model, Opt.Scalar == Float,
                               Model.Input == Tensor<Float>,
                               Model.Output == Tensor<Float>
{
    for batch in train_ds{
        let (loss, grads) = model.valueWithGradient { model -> Tensor<Float> in
            let preds = model.applied(to: batch.x, in: trainingContext)
            return loss_func(preds, batch.y)
        }
        print(loss)
        opt.update(&model.allDifferentiableVariables, along: grads)
    }
}

: 

In [ ]:
func basic_training_loop<Model, Opt:Optimizer> (train_ds:Dataset<Batch>, model: inout Model, opt: inout Opt, 
                         loss_func: (Tensor<Float>, Tensor<Float>)->Tensor<Float>)
                         where Opt.Model == Model, Opt.Scalar == Float,
                               Model.Input == Tensor<Float>,
                               Model.Output == Tensor<Float>
{
    for batch in train_ds{
        let (loss, grads) = model.valueWithGradient(at: batch.x) { (model, y) -> Tensor<Float> in
            let preds = model.applied(to: batch.x, in: trainingContext)
            return loss_func(preds, y)
        }
        print(loss)
        opt.update(&model.allDifferentiableVariables, along: grads)
    }
}

: 